In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from adapt.instance_based import TrAdaBoostR2
from lightgbm import LGBMRegressor as lgbm
from sklearn import metrics

In [2]:
yields = pd.read_csv('data/Yield_list.csv')
data_s_woyields = pd.read_csv('data/data_s_woyields.csv')
data_t = pd.read_csv('data/data_t.csv')
yt = pd.DataFrame(data_t['Yield'],columns=['Yield'])
Xt = data_t.drop(columns=['Name', 'ID', 'Yield'])

In [3]:
r2_unlab = []
selected_SD = []
for i in range(100):
    yield_lab, yield_unlab = train_test_split(yields, test_size=0.5, random_state=i)
    yield_lab_ID = pd.DataFrame(yield_lab['ID'],columns=['ID'])
    yield_lab_values = yield_lab.drop(columns=['Name', 'ID'])
    
    corr_matrix = yield_lab_values.corr()
    corr = corr_matrix.loc[['CA'], :]
    corr = np.array(corr)
    columns = ['CO_a', 'CO_b', 'CO_c', 'CO_d', 'CO_e', 'CS', 'CN', 'CA']
    corr = pd.DataFrame(data=corr, columns=columns)
    bottom_3 = corr.drop(columns=['CA']).T[0].nsmallest(3).index.tolist()
    selected_SD.append(bottom_3)

    Yield = []
    for dataset in bottom_3:
        reaction_yield = pd.read_csv(f'data/Yield_{dataset}.csv')
        Yield.append(reaction_yield)

    concat_yield = pd.concat(Yield, axis=0, ignore_index=True)
    data_s = pd.concat([data_s_woyields, concat_yield], axis=1, join='inner')
    ys = pd.DataFrame(data_s['Yield'],columns=['Yield'])
    Xs = data_s.drop(columns=['Name', 'ID', 'Yield'])
    
    Xt_lab, Xt_unlab, yt_lab, yt_unlab = train_test_split(Xt, yt, test_size=0.5, random_state=i)
    model = TrAdaBoostR2(lgbm(), n_estimators=5, Xt=Xt_lab, yt=yt_lab, random_state=0, verbose=0)
    model.fit(Xs, ys)
    y_pred2 = model.predict(Xt_unlab)
    r2_unlab.append(metrics.r2_score(yt_unlab, y_pred2))

selectedSD = pd.DataFrame(selected_SD, columns=['SD1', 'SD2', 'SD3'])
result = pd.DataFrame(r2_unlab, columns=['R2_score'])
result = pd.concat([selectedSD, result], axis=1, join='inner')
result.to_csv('result/bottom/result.csv')

print('Avg_R2_test:', np.mean(r2_unlab))
print('Max_R2_test:', np.max(r2_unlab))
print('Std_R2_test:', np.std(r2_unlab, ddof=0))

Avg_R2_test: 0.5722771732251108
Max_R2_test: 0.812544117474377
Std_R2_test: 0.11605562719470561
